In [1]:
import itertools
import torch
import pickle
import numpy as np
import ZH_Nakamura
import subprocess
import os

torch.set_default_dtype(torch.float64)
import sys
sys.path.append('..')
from Tools import syncer 
from Tools import user
from Tools import helpers
import ROOT

Welcome to JupyROOT 6.24/06


In [2]:
def make_weight_ratio(r, weights, base_points):
    output=0
    nCoefficients=len(r.coefficient_names)
    row,column=np.triu_indices(nCoefficients+1)
    i, j = np.triu_indices(8)                                                                                       
    for i_comb, combination in enumerate(r.combination_list):
        output+=torch.outer(torch.from_numpy(weights[r.combination_list[i_comb]]/weights[()]),(base_points[:,row[i_comb]]*base_points[:,column[i_comb]]))
    return output

In [3]:
def f_loss(r,features,base_points,weights):
        fhat = 1./(1. + r.predict_r_hat(features,base_points))
        weight_ratios=make_weight_ratio(r, weights, base_points)
        loss = (torch.from_numpy(weights[()])*torch.transpose(weight_ratios*fhat**2 + (1-fhat)**2,0,1)).sum()
        return loss

In [4]:
class R:
    def __init__(self,nfeatures,coefficient_names):
        self.nfeatures         = nfeatures
        self.coefficient_names = coefficient_names
        self.combination_list=list(itertools.chain.from_iterable(itertools.combinations_with_replacement(self.coefficient_names, i) for i in np.arange(0,3)))
        self.n_hat = {combination: self.make_NN() for combination in self.combination_list}
        
    def make_NN(self, hidden_layers  = [32, 32, 32, 32]):
        model_nn = [torch.nn.BatchNorm1d(self.nfeatures), torch.nn.ReLU(), torch.nn.Linear(self.nfeatures, hidden_layers[0])]
        for i_layer, layer in enumerate(hidden_layers):
            model_nn.append(torch.nn.Linear(hidden_layers[i_layer], hidden_layers[i_layer+1] if i_layer+1<len(hidden_layers) else 1))
            if i_layer+1<len(hidden_layers):
                model_nn.append( torch.nn.ReLU() )
        return torch.nn.Sequential(*model_nn)

    def evaluate_NN(self, features):
        '''Evaluate Neural Network: The zeroth dimension of features is the number of data points and and the first dimension
        is the number of features(variables)
        '''
        noutputs=len(self.combination_list)
        ndatapoints=features.shape[0]
        
        output=torch.zeros((noutputs,ndatapoints))
        for i in range(noutputs):
            #self.n_hat[self.combination_list[i]].eval()
            x=self.n_hat[self.combination_list[i]](features)
            if i==0:
                output[i,:]=1
            else:
                output[i,:]=torch.flatten(x)            
        return output
        
    
    def predict_r_hat(self, features, base_points):
        '''Evaluate positive xsec ratio for given theta and 
        '''
        ndatapoints=features.shape[0]
        output_NN = self.evaluate_NN(features)
        n_terms=len(self.coefficient_names)
        row,column=np.triu_indices(n_terms+1)
        Omega=torch.zeros((n_terms+1,n_terms+1,ndatapoints))
        for i in range(0, len(row)):
            Omega[row[i]][column[i]][:]=output_NN[i,:]
        Omega_swapped=torch.swapaxes(Omega,1,2)
        Omega_swapped=torch.swapaxes(Omega_swapped,0,1)
        
        out=torch.matmul(Omega_swapped,torch.transpose(base_points,0,1))
        return torch.linalg.norm(out, 2, 1)
    
    def save(self,fileName):
        outfile = open(fileName,'wb')
        pickle.dump(self, outfile)
        outfile.close()
        
    @classmethod
    def load(self, fileName):
        infile = open(fileName,'rb')
        print(fileName)
        new_dict = pickle.load(infile)
        infile.close()
        return new_dict

In [5]:
n_features=6
#coefficients=['cHQ3', 'cHW', 'cHWtil']
#base_points = np.asarray([np.array([1, value1, value2, value3]) for value1 in [-1.5, -.8, .2, 0., .2, .8, 1.5]  for value2 in [-1.5, -.8, .2, 0, .2, .8, 1.5] for value3 in [-1.5, -.8, .2, 0, .2, .8, 1.5]])
#coefficients=['cHW', 'cHWtil']
#base_points = np.asarray([np.array([1, value1, value2]) for value1 in [-1.5, -.8, .2, 0., .2, .8, 1.5]  for value2 in [-1.5, -.8, .2, 0, .2, .8, 1.5]])
coefficients=['cHQ3']
base_points = np.asarray([np.array([1, value1]) for value1 in [-1.5, -.8, .2, 0., .2, .8, 1.5]])
print(base_points.shape)
r_NN = R(n_features, coefficients)
device = 'cuda' if torch.cuda.is_available() else 'cpu'

(7, 2)


In [6]:
plot_directory="v4_Class_7"

nEvents=30000
learning_rate = 1e-3
device        = 'cuda' if torch.cuda.is_available() else 'cpu'
n_epoch       = 10000
plot_every    = 100

In [7]:
# training data
import ZH_Nakamura 
ZH_Nakamura.feature_names = ZH_Nakamura.feature_names[0:6] # restrict features
features   = ZH_Nakamura.getEvents(nEvents)[:,0:6]
feature_names  = ZH_Nakamura.feature_names
plot_options   = ZH_Nakamura.plot_options
plot_vars      = ZH_Nakamura.feature_names

mask       = (features[:,feature_names.index('pT')]<900) & (features[:,feature_names.index('sqrt_s_hat')]<1800) 
features = features[mask]

n_features = len(features[0])
weights    = ZH_Nakamura.getWeights(features, ZH_Nakamura.make_eft())


WC='cHQ3'

pT=features[:,feature_names.index('pT')]
features=torch.from_numpy(features)
#w0_train       = torch.from_numpy(weights[()]).float().to(device)
#wp_train       = torch.from_numpy(weights[(WC,)]).float().to(device)
#wpp_train      = torch.from_numpy(weights[(WC,WC)]).float().to(device)
w0_train       = weights[()]
wp_train       = weights[(WC,)]
wpp_train      = weights[(WC,WC)]




Requested 30000 events. Simulated 30000 events and 30000 survive pT_min cut of 0.


In [8]:
all_params=[]
for comb in r_NN.combination_list:
        all_params+=r_NN.n_hat[comb].parameters()

In [9]:
#optimizer = torch.optim.RMSprop(model.parameters(), lr=learning_rate)
optimizer = torch.optim.Adam(list(all_params), lr=learning_rate)
losses = []
losses_train=[]
for comb in r_NN.combination_list:
    r_NN.n_hat[comb].train()

In [10]:
tex = ROOT.TLatex()
tex.SetNDC()
tex.SetTextSize(0.04)

In [11]:
for epoch in range(n_epoch):
    loss = f_loss(r_NN,features,torch.from_numpy(base_points),weights)
    print("epoch", epoch, "loss outside loop",  loss.item())
    
    losses.append(loss.item())
    optimizer.zero_grad()    
    loss.backward()
    optimizer.step()
    
    '''if (epoch % plot_every)==0:
        with torch.no_grad():
            pred_p = r_NN.n_hat[('cHQ3',)](features).squeeze().cpu().detach().numpy()
            pred_pp = r_NN.n_hat[('cHQ3', 'cHQ3')](features).squeeze().cpu().detach().numpy()
            predicted_weights={():w0_train, ('cHQ3',):pred_p, ('cHQ3', 'cHQ3'):pred_pp}
            
            loss_train = f_loss(r_NN, features, torch.from_numpy(base_points), predicted_weights)
            losses_train.append(loss_train.item())
            
            print("epoch", epoch, "loss inside loop",  loss_train.item())
                       
            for var in plot_vars:
                binning     = plot_options[var]['binning']
                np_binning  = np.linspace(binning[1], binning[2], 1+binning[0])

                
                truth_0  = np.histogram(features[:,feature_names.index(var)], np_binning, weights=w0_train )
                truth_p  = np.histogram(features[:,feature_names.index(var)], np_binning, weights=wp_train )
                truth_pp = np.histogram(features[:,feature_names.index(var)], np_binning, weights=wpp_train )

                pred_0_hist  = np.histogram(features[:,feature_names.index(var)], np_binning, weights=w0_train)
                pred_p_hist  = np.histogram(features[:,feature_names.index(var)], np_binning, weights=w0_train*pred_p)
                pred_pp_hist = np.histogram(features[:,feature_names.index(var)], np_binning, weights=w0_train*pred_pp)

                h_yield       = helpers.make_TH1F(truth_0)
                h_truth_p     = helpers.make_TH1F(truth_p)
                h_truth_p     .Divide(h_yield) 
                h_truth_pp    = helpers.make_TH1F(truth_pp)
                h_truth_pp    .Divide(h_yield) 

                h_pred_p      = helpers.make_TH1F(pred_p_hist)
                h_pred_p      .Divide(h_yield) 
                h_pred_pp     = helpers.make_TH1F(pred_pp_hist)
                h_pred_pp     .Divide(h_yield) 

                l = ROOT.TLegend(0.3,0.7,0.9,0.95)
                l.SetNColumns(2)
                l.SetFillStyle(0)
                l.SetShadowColor(ROOT.kWhite)
                l.SetBorderSize(0)

                h_yield      .SetLineColor(ROOT.kGray+2) 
                h_truth_p    .SetLineColor(ROOT.kBlue) 
                h_truth_pp   .SetLineColor(ROOT.kRed) 
                h_pred_p     .SetLineColor(ROOT.kBlue) 
                h_pred_pp    .SetLineColor(ROOT.kRed) 
                h_yield      .SetMarkerColor(ROOT.kGray+2) 
                h_truth_p    .SetMarkerColor(ROOT.kBlue) 
                h_truth_pp   .SetMarkerColor(ROOT.kRed) 
                h_pred_p     .SetMarkerColor(ROOT.kBlue) 
                h_pred_pp    .SetMarkerColor(ROOT.kRed) 
                h_yield      .SetMarkerStyle(0)
                h_truth_p    .SetMarkerStyle(0)
                h_truth_pp   .SetMarkerStyle(0)
                h_pred_p     .SetMarkerStyle(0)
                h_pred_pp    .SetMarkerStyle(0)

                l.AddEntry(h_truth_p   , "1^{st.} der (truth)" ) 
                l.AddEntry(h_truth_pp  , "2^{st.} der (truth)" ) 
                l.AddEntry(h_pred_p    , "1^{st.} der (pred)" ) 
                l.AddEntry(h_pred_pp   , "2^{st.} der (pred)" ) 
                l.AddEntry(h_yield     , "yield" ) 

                h_truth_p    .SetLineStyle(ROOT.kDashed) 
                h_truth_pp   .SetLineStyle(ROOT.kDashed)

                lines = [ 
                        (0.16, 0.965, 'Epoch %5i    Loss %6.4f'%( epoch, loss ))
                        ]

                max_ = max( map( lambda h:h.GetMaximum(), [ h_truth_p, h_truth_pp ] ))

                h_yield.Scale(max_/h_yield.GetMaximum())
                for logY in [True, False]:
                    c1 = ROOT.TCanvas()
                    h_yield   .Draw("hist")
                    h_yield   .GetYaxis().SetRangeUser(0.1 if logY else 0, 10**(1.5)*max_ if logY else 1.5*max_)
                    h_yield   .Draw("hist")
                    h_truth_p .Draw("hsame") 
                    h_truth_pp.Draw("hsame")
                    h_pred_p  .Draw("hsame") 
                    h_pred_pp .Draw("hsame")
                    c1.SetLogy(logY) 
                    l.Draw()

                    drawObjects = [ tex.DrawLatex(*line) for line in lines ]
                    for o in drawObjects:
                        o.Draw()

                    plot_directory_final = os.path.join(plot_directory, "log" if logY else "lin")
                    helpers.copyIndexPHP( plot_directory )
                    c1.Print( os.path.join( plot_directory, "epoch_%05i_%s.png"%(epoch, var) ) )
                    '''

epoch 0 loss outside loop 2.1027393494775173
epoch 1 loss outside loop 2.0950571602294374
epoch 2 loss outside loop 2.0876983784153778
epoch 3 loss outside loop 2.0806133279214074
epoch 4 loss outside loop 2.0737200547741845
epoch 5 loss outside loop 2.066965675490178
epoch 6 loss outside loop 2.0602082556285826
epoch 7 loss outside loop 2.0533511678653196
epoch 8 loss outside loop 2.046320468295894
epoch 9 loss outside loop 2.038990097911599
epoch 10 loss outside loop 2.031406261730531
epoch 11 loss outside loop 2.023530056559137
epoch 12 loss outside loop 2.0153325330862892
epoch 13 loss outside loop 2.0067652418952533
epoch 14 loss outside loop 1.9977615349003544
epoch 15 loss outside loop 1.9882523057810815
epoch 16 loss outside loop 1.978173004824899
epoch 17 loss outside loop 1.967428461639244
epoch 18 loss outside loop 1.9559320122389676
epoch 19 loss outside loop 1.9435590381176895
epoch 20 loss outside loop 1.9301793597570032
epoch 21 loss outside loop 1.9156716011044006
epoch

KeyboardInterrupt: 